# Web Scraping con Selenium

##### Ejercicio en grupos

^Daniel Vivas - Data Science Teacher Assistant - DS Abril 2022^

##### En parejas, vamos a buscar las cuatro primeras películas en cartelera. De ellas vamos a extraer la siguiente información:
- ##### Fecha de estreno
- ##### URL
- ##### Datos principales, como hemos hecho al principio
- ##### Nota media
- ##### Cantidad de votos
- ##### Críticas profesionales buenas, regulares y malas
- ##### Cinco primeras críticas

In [2]:
#%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 6.7/6.7 MB 10.7 MB/s eta 0:00:00
     -------------------------------------- 399.3/399.3 kB 8.3 MB/s eta 0:00:00
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
#%pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Importación de las librerías

In [6]:
#Para la manipulación de datos
import pandas as pd

#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para esperar a que carguen las páginas
import time


Creamos el driver para controlar el navegador

In [7]:
service=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 9.23MB/s]


RECUERDA:

##### Beginner Selenium Cheatsheet:
Sacar un elemento:
- element = driver.find_element(by, value)

Sacar varios elementos:
- element = driver.find_elements(by, value)

Sacar atributos:
- attribute = element.--el atributo--
- attribute = element.get_attribute(--el atributo--)

Hacer click:
- element.click()

Teclear:
- element.send_keys()

Gestión de pestañas:
- driver.switch_to.window(driver.window_handles[-1])
- driver.get(url)
- driver.close()

Accedemos a la página principal

In [8]:
driver.get("https://www.filmaffinity.com/")

Aceptamos el pop-up de ser necesario

In [9]:
#Seleccionamos el botón
accept = driver.find_element(by=By.CLASS_NAME, value='css-v43ltw')

#Hacemos click
accept.click()

Hacemos una función que devuelva en un diccionario todos los datos de las películas, salvo la fecha de estreno y la url

Parámetros: url y fecha de estreno
Salida: Diccionario con todos los datos

In [10]:
def get_datos_pelicula(driver, url, estreno):

    #Creamos el diccionario
    movie_dict = dict()

    #Añadimos al diccionario los parámetros que nos vienen dados
    movie_dict['fecha_estreno'] = estreno
    movie_dict['url'] = url

    #Accedemos a la url
    driver.get(url)
    time.sleep(2)

    #Sacamos los datos principales
    principales = driver.find_element(by=By.CLASS_NAME, value='movie-info')

    dts = principales.find_elements(by=By.TAG_NAME, value='dt')
    dds = principales.find_elements(by=By.TAG_NAME, value='dd')

    i = 0
    while i < len(dts):
        movie_dict[dts[i].text] = dds[i].text
        i += 1

    #Sacamos la nota media
    nota_media = driver.find_elements(by=By.ID, value="movie-rat-avg")
    if len(nota_media) > 0:
        movie_dict['nota_media'] = nota_media[0].text

    #Sacamos la cantidad de votos
    cantidad_votos = driver.find_elements(by=By.CSS_SELECTOR, value="#movie-count-rat span")
    if len(cantidad_votos) > 0:
        movie_dict['cantidad_votos'] = cantidad_votos[0].text

    #Críticas profesionales
    n_criticas = driver.find_elements(by=By.CSS_SELECTOR, value='#right-column > a > div > div.body > div > div.legend-wrapper .leg')

    if len(n_criticas) > 0:
        #Criticas profesionales positivas
        positivas = n_criticas[0].text
        movie_dict['positivas'] = positivas

        #Criticas profesionales regulares
        regulares = n_criticas[1].text
        movie_dict['regulares'] = regulares

        #Criticas profesionales negativas
        negativas = n_criticas[2].text
        movie_dict['negativas'] = negativas

    #Cinco primeras criticas
    criticas = driver.find_elements(by=By.CSS_SELECTOR, value="ul#pro-reviews li")

    i = 0

    while i < 5 and i < len(criticas):
        critica = criticas[i].find_element(by=By.CSS_SELECTOR, value='div div').text
        movie_dict['critica_'+str(i)] = critica

        i += 1


    return movie_dict


Probamos la función que hemos hecho. Aquí tienes un enlace de prueba: https://www.filmaffinity.com/es/film599984.html

In [11]:
prueba = get_datos_pelicula(driver, "https://www.filmaffinity.com/es/film618375.html", "fecha")
prueba

{'fecha_estreno': 'fecha',
 'url': 'https://www.filmaffinity.com/es/film618375.html',
 'Título original': 'Oblivion',
 'Año': '2013',
 'Duración': '126 min.',
 'País': ' Estados Unidos',
 'Dirección': 'Joseph Kosinski',
 'Guion': 'Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson',
 'Reparto': 'Tom Cruise\nAndrea Riseborough\nOlga Kurylenko\nMorgan Freeman\nNikolaj Coster-Waldau\nZoe Bell',
 'Música': 'Anthony Gonzalez, M83, Joseph Trapanese',
 'Fotografía': 'Claudio Miranda',
 'Compañías': 'Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios',
 'Género': 'Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic',
 'Sinopsis': 'Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es uno de los últimos hombres que la habitan. Es un ingeniero de Dron

Entramos en el link de las películas en cartelera

In [12]:
driver.find_elements(By.PARTIAL_LINK_TEXT, value="Películas en cartelera")[0].click()

Sacamos todas las películas y llamamos a la función con cada película

In [13]:
#Creamos una lista vacia
links = []

#Sacamos el elemento raíz
filas = driver.find_elements(by=By.CSS_SELECTOR, value='#wrapper-cat div.row')

for fila in filas:

    peliculas = fila.find_elements(by=By.CLASS_NAME, value='movie-poster')

    for pelicula in peliculas:
        url = pelicula.find_element(By.TAG_NAME, value='a').get_attribute('href')
        estreno = pelicula.find_element(By.CSS_SELECTOR, value='a div.release-text').text.replace('\n', ' ')

        link_pelicula = {
            'url': url,
            'estreno': estreno
        }

        links.append(link_pelicula)


Ahora usamos los links para llamar a la funcion y sacar los datos

In [14]:
#Reservamos la variable para el dataframe
df = None

for link in links:
    datos_pelicula = get_datos_pelicula(driver, link['url'], link['estreno'])

    #Si no está creado lo creamos
    if df is None:
        df = pd.DataFrame(columns=datos_pelicula.keys())

    #Metemos los datos de la película
    df = df.append(datos_pelicula, ignore_index=True)
    print(f"Añadida {datos_pelicula['Título original']}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_24632\3694728036.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datos_pelicula, ignore_index=True)


Añadida Vacaciones de verano


C:\Users\Admin\AppData\Local\Temp\ipykernel_24632\3694728036.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datos_pelicula, ignore_index=True)


Añadida Bed Rest


C:\Users\Admin\AppData\Local\Temp\ipykernel_24632\3694728036.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datos_pelicula, ignore_index=True)


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_24632\3694728036.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datos_pelicula, ignore_index=True)


Añadida ¡Vaya vacaciones!aka


C:\Users\Admin\AppData\Local\Temp\ipykernel_24632\3694728036.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datos_pelicula, ignore_index=True)


Añadida 20.000 especies de abejas
Añadida The Super Mario Bros. Movie


C:\Users\Admin\AppData\Local\Temp\ipykernel_24632\3694728036.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(datos_pelicula, ignore_index=True)
